In [1]:
import torch

import cv2 as cv
import numpy as np

from pytorch3d.io import load_obj
from pytorch3d.ops import interpolate_face_attributes
from pytorch3d.structures import Meshes
from pytorch3d.renderer.mesh.textures import TexturesUV

from pytorch3d.renderer.cameras import PerspectiveCameras, look_at_view_transform
from pytorch3d.renderer.mesh import MeshRasterizer, RasterizationSettings

import torchvision.transforms.functional as tvf

import utils.io as io

base_path = 'dataset/test_data/'

# load mesh
verts, faces, aux = load_obj(base_path + 'OBJs/Head.obj', device='cuda')

In [67]:
def load_sdr(image_name):
    
    image = cv.imread(image_name, cv.IMREAD_UNCHANGED)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    
    if image.dtype == 'uint8':
        image = image / 255.
    elif image.dtype == 'uint16':
        image = image / 65535.
    
    image = cv.resize(image, (4096, 4096), interpolation=cv.INTER_NEAREST)
    
    return torch.from_numpy(image).float()

# load textures
basecolor = load_sdr(base_path + f'/Textures/Albedo.png').cuda()
roughness = load_sdr(base_path + f'/Textures/Roughness_CR.png').cuda()
specular = load_sdr(base_path + f'/Textures/Specular_CC.png').cuda()
#normal = load_sdr(base_path + f'/Textures/Normal.png').cuda()
#normal = torch.nn.functional.normalize(normal, dim=-1)

verts_uvs = aux.verts_uvs.cuda()
faces_uvs = faces.textures_idx.cuda()
image = torch.cat([basecolor, roughness[...,0][..., None], specular[...,0][..., None]], dim=-1)[None]
texture = TexturesUV(verts_uvs=[verts_uvs], faces_uvs=[faces_uvs], maps=image)

mesh = Meshes(verts=[verts], faces=[faces.verts_idx], textures=texture).to('cuda')
camera = PerspectiveCameras(in_ndc=False, image_size=[(512, 512)], device='cuda')
rasterizer_settings = RasterizationSettings(image_size=512)
rasterizer = MeshRasterizer(raster_settings=rasterizer_settings, cameras=camera)
rasterizer = rasterizer.to('cuda')

In [89]:
from utils.transforms import B2P

# RT = torch.tensor([[1.0, 0.0, 0.0, 0.0],
#                    [0.0, -0.052335940301418304, -0.9986295104026794, -0.699999988079071],
#                    [0.0, 0.9986295104026794, -0.052335940301418304, 0.800000011920929],
#                    [0.0, 0.0, 0.0, 1.0]]).cuda()

RT = torch.tensor([[0.0, 0.0, 0.0, 0.0],
                   [0.0, -0.052335940301418304, -0.9986295104026794, -0.699999988079071],
                   [0.0, 0.9986295104026794, -0.052335940301418304, 0.800000011920929],
                   [0.0, 0.0, 0.0, 1.0]]).cuda()

R, T, RT_4x4 = B2P(RT)

#R, T = look_at_view_transform(dist=1.5, elev=-2, azim=3.1415, device='cuda') 

K = torch.tensor([[512., 0., 256., 0.],
                  [0., 512., 256., 0.],
                  [0., 0., 0., 1.],
                  [0., 0., 1., 0.]]).cuda()

with torch.no_grad():
    
    rasterizer.cameras.R = R[None]
    rasterizer.cameras.T = T[None]
    rasterizer.cameras.K = K[None]
    
    fragments = rasterizer(mesh)

    textures = mesh.textures.sample_textures(fragments).squeeze(3)

tvf.to_pil_image(textures[0][...,:3].permute(2,0,1)).save('test.png')

Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size to 0 to use the naive rasterization.Bin size was too small in the coarse rasterization phase. This caused an overflow, meaning output may be incomplete. To solve, try increasing max_faces_per_bin / max_points_per_bin, decreasing bin_size, or setting bin_size 